# EX04 Solutions
Solutions to the fourth week's exercises on camera calibration.

In [1]:
import numpy as np
from scipy.spatial.transform import Rotation

### Exercise 4.1
Using the pinhole camera model, the projection matrix is given by $P=K[R\mid t]$. We project the cube corners $Q$ (with $i,j,k\in\{0,1\}$) using homogeneous coordinates.

In [2]:
# World points
Q = np.array([[i,j,k] for i in [0,1] for j in [0,1] for k in [0,1]]).T

# Intrinsic matrix for f=1000 and 1920x1080 sensor
f = 1000
width, height = 1920, 1080
K = np.array([[f,0,width/2],[0,f,height/2],[0,0,1]])

# Camera pose
s = np.sqrt(1/2)
R = np.array([[s,-s,0],[s,s,0],[0,0,1]])
t = np.array([0,0,10])

# Projection matrix and image projections
P = K @ np.hstack((R, t.reshape(3,1)))
Qh = np.vstack((Q, np.ones((1,Q.shape[1]))))
qh = P @ Qh
q = qh[:2] / qh[2]
print('Projection matrix P:', P)
print('Image points q:', q)

Projection matrix P: [[ 7.07106781e+02 -7.07106781e+02  9.60000000e+02  9.60000000e+03]
 [ 7.07106781e+02  7.07106781e+02  5.40000000e+02  5.40000000e+03]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  1.00000000e+01]]
Image points q: [[ 960.          960.          889.28932188  895.71756535 1030.71067812
  1024.28243465  960.          960.        ]
 [ 540.          540.          610.71067812  604.28243465  610.71067812
   604.28243465  681.42135624  668.56486931]]


### Exercise 4.2
The Direct Linear Transform (DLT) stacks the equations $\mathbf{q}	imes(P\mathbf{Q})=0$ and solves for $P$ via SVD. We estimate $P$ with and without normalizing the image points and report the reprojection error.

In [3]:
def normalize2d(points):
    centroid = points.mean(axis=1, keepdims=True)
    shifted = points - centroid
    mean_dist = np.mean(np.sqrt(np.sum(shifted**2, axis=0)))
    scale = np.sqrt(2) / mean_dist
    T = np.array([[scale,0,-scale*centroid[0,0]],[0,scale,-scale*centroid[1,0]],[0,0,1]])
    pts_h = np.vstack((points, np.ones(points.shape[1])))
    pts_norm = T @ pts_h
    return T, pts_norm[:2]

def pest(Q, q, normalize=False):
    if normalize:
        T, qn = normalize2d(q)
    else:
        T = np.eye(3); qn = q
    A = []
    for i in range(Q.shape[1]):
        X,Y,Z = Q[:,i]
        u,v = qn[:,i]
        A.append([X,Y,Z,1,0,0,0,0,-u*X,-u*Y,-u*Z,-u])
        A.append([0,0,0,0,X,Y,Z,1,-v*X,-v*Y,-v*Z,-v])
    A = np.array(A)
    _,_,Vt = np.linalg.svd(A)
    Pn = Vt[-1].reshape(3,4)
    P = np.linalg.inv(T) @ Pn
    return P

P_est = pest(Q, q)
Qh = np.vstack((Q, np.ones((1,Q.shape[1]))))
q_est = P_est @ Qh
q_est = q_est[:2] / q_est[2]
rmse = np.sqrt(np.mean(np.sum((q_est - q)**2, axis=0)))

P_est_n = pest(Q, q, normalize=True)
q_est_n = P_est_n @ Qh
q_est_n = q_est_n[:2] / q_est_n[2]
rmse_n = np.sqrt(np.mean(np.sum((q_est_n - q)**2, axis=0)))

print('RMSE without normalization:', rmse)
print('RMSE with normalization   :', rmse_n)

RMSE without normalization: 6.268222281955226e-11
RMSE with normalization   : 1.503935499763765e-13


### Exercise 4.3
`checkerboard_points` generates an $n	imes m$ grid of planar points centered at the origin.

In [4]:
def checkerboard_points(n, m):
    i, j = np.meshgrid(np.arange(n), np.arange(m), indexing='ij')
    coords = np.vstack((i.ravel()-(n-1)/2, j.ravel()-(m-1)/2, np.zeros(n*m)))
    return coords

Q_omega = checkerboard_points(10, 20)

### Exercise 4.4
We create three checkerboards by rotating $Q_{\omega}$ about the $x$-axis and project them to the image plane.

In [5]:
rot = Rotation.from_euler('xyz', [np.pi/10, 0, 0]).as_matrix()
Qa = rot @ Q_omega
Qb = Q_omega.copy()
rot_neg = Rotation.from_euler('xyz', [-np.pi/10, 0, 0]).as_matrix()
Qc = rot_neg @ Q_omega

def project(P, Q):
    Qh = np.vstack((Q, np.ones((1, Q.shape[1]))))
    qh = P @ Qh
    return qh[:2] / qh[2]

qa = project(P, Qa)
qb = project(P, Qb)
qc = project(P, Qc)

### Exercise 4.5
Homographies mapping $Q_{\omega}$ to each set of image points are estimated using the DLT homography estimator `hest`.

In [6]:
def hest(q1, q2, normalize=True):
    if normalize:
        T1, q1n = normalize2d(q1)
        T2, q2n = normalize2d(q2)
    else:
        T1 = np.eye(3); q1n = q1
        T2 = np.eye(3); q2n = q2
    A = []
    for i in range(q1.shape[1]):
        x, y = q2n[:, i]
        u, v = q1n[:, i]
        A.append([-x, -y, -1, 0, 0, 0, u*x, u*y, u])
        A.append([0, 0, 0, -x, -y, -1, v*x, v*y, v])
    A = np.array(A)
    _,_,Vt = np.linalg.svd(A)
    Hn = Vt[-1].reshape(3,3)
    H = np.linalg.inv(T1) @ Hn @ T2
    return H / H[2,2]


def estimateHomographies(Q_omega, qs):
    Q2d = Q_omega[:2]
    return [hest(q, Q2d, normalize=True) for q in qs]

Hs = estimateHomographies(Q_omega, [qa, qb, qc])
Q2dh = np.vstack((Q_omega[:2], np.ones(Q_omega.shape[1])))
for H, q in zip(Hs, [qa, qb, qc]):
    qh = H @ Q2dh
    q_proj = qh[:2] / qh[2]
    err = np.sqrt(np.mean(np.sum((q_proj - q)**2, axis=0)))
    print('homography RMSE:', err)

homography RMSE: 3.5367659221829735e-13
homography RMSE: 2.931056608759553e-13
homography RMSE: 3.8221874695231336e-13


### Exercise 4.6
From the homographies we form the matrix $V$ and solve $V\mathbf{b}=0$ to obtain the vector $\mathbf{b}$ representing $B = K^{-T}K^{-1}$.

In [7]:
def v_ij(H, i, j):
    h = H.T
    return np.array([
        h[i,0]*h[j,0],
        h[i,0]*h[j,1] + h[i,1]*h[j,0],
        h[i,1]*h[j,1],
        h[i,2]*h[j,0] + h[i,0]*h[j,2],
        h[i,2]*h[j,1] + h[i,1]*h[j,2],
        h[i,2]*h[j,2]])

def estimate_b(Hs):
    V = []
    for H in Hs:
        V.append(v_ij(H,0,1))
        V.append(v_ij(H,0,0) - v_ij(H,1,1))
    V = np.array(V)
    _,_,Vt = np.linalg.svd(V)
    return Vt[-1]

b = estimate_b(Hs)
invK = np.linalg.inv(K)
B = invK.T @ invK
b_true = np.array([B[0,0], B[0,1], B[1,1], B[0,2], B[1,2], B[2,2]])
scale = b[0] / b_true[0]
print('b estimate:', b)
print('b_true scaled:', scale * b_true)

b estimate: [-4.51834392e-07  1.89336415e-22 -4.51834392e-07  4.33761016e-04
  2.43990572e-04 -9.99999876e-01]
b_true scaled: [-4.51834392e-07 -0.00000000e+00 -4.51834392e-07  4.33761016e-04
  2.43990572e-04 -9.99999876e-01]


### Exercise 4.7
The intrinsic parameters are recovered from $\mathbf{b}$, yielding the camera matrix $K$.

In [8]:
def estimateIntrinsics(Hs):
    b = estimate_b(Hs)
    B11,B12,B22,B13,B23,B33 = b
    v0 = (B12*B13 - B11*B23) / (B11*B22 - B12**2)
    lambda_ = B33 - (B13**2 + v0*(B12*B13 - B11*B23)) / B11
    alpha = np.sqrt(lambda_ / B11)
    beta = np.sqrt(lambda_ * B11 / (B11*B22 - B12**2))
    gamma = -B12 * alpha**2 * beta / lambda_
    u0 = gamma * v0 / beta - B13*alpha**2 / lambda_
    return np.array([[alpha, gamma, u0],[0, beta, v0],[0,0,1]])

K_est = estimateIntrinsics(Hs)
print('Estimated K:', K_est)

Estimated K: [[1.00000000e+03 4.19039404e-13 9.60000000e+02]
 [0.00000000e+00 1.00000000e+03 5.40000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


### Exercise 4.8
Extrinsic parameters for each view are obtained from the homographies. Combining the previous steps yields a `calibratecamera` function.

In [9]:
def estimateExtrinsics(K, Hs):
    invK = np.linalg.inv(K)
    Rs, ts = [], []
    for H in Hs:
        h1, h2, h3 = H[:,0], H[:,1], H[:,2]
        lam = 1/np.linalg.norm(invK @ h1)
        r1 = lam * (invK @ h1)
        r2 = lam * (invK @ h2)
        r3 = np.cross(r1, r2)
        R = np.column_stack((r1, r2, r3))
        U,_,Vt = np.linalg.svd(R)
        R = U @ Vt
        t = lam * (invK @ h3)
        Rs.append(R); ts.append(t)
    return Rs, ts

def calibratecamera(qs, Q):
    Hs = estimateHomographies(Q, qs)
    K = estimateIntrinsics(Hs)
    Rs, ts = estimateExtrinsics(K, Hs)
    return K, Rs, ts

K_c, Rs_c, ts_c = calibratecamera([qa, qb, qc], Q_omega)
print('Calibrated K:', K_c)

Calibrated K: [[1.00000000e+03 4.19039404e-13 9.60000000e+02]
 [0.00000000e+00 1.00000000e+03 5.40000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


### Exercise 4.9
We reproject the checkerboards using the estimated parameters and compute the root mean squared reprojection error.

In [10]:
def project_with_RT(K, R, t, Q):
    P = K @ np.hstack((R, t.reshape(3,1)))
    Qh = np.vstack((Q, np.ones((1, Q.shape[1]))))
    qh = P @ Qh
    return qh[:2] / qh[2]

for idx, (R, t, q) in enumerate(zip(Rs_c, ts_c, [qa, qb, qc])):
    q_proj = project_with_RT(K_c, R, t, Q_omega)
    err = np.sqrt(np.mean(np.sum((q_proj - q)**2, axis=0)))
    print(f'view {idx} RMSE: {err}')

view 0 RMSE: 5.687583751789392e-13
view 1 RMSE: 3.8290046402730916e-13
view 2 RMSE: 3.9363052515600186e-13


### Exercise 4.10
Adding Gaussian noise with $\sigma=1$ px to the image points still yields an almost correct calibration.

In [11]:
noise_a = qa + np.random.normal(0,1,qa.shape)
noise_b = qb + np.random.normal(0,1,qb.shape)
noise_c = qc + np.random.normal(0,1,qc.shape)
K_noisy, Rs_noisy, ts_noisy = calibratecamera([noise_a, noise_b, noise_c], Q_omega)
print('K difference (noisy - true):', K_noisy - K)

K difference (noisy - true): [[-2.86023913 -0.07937287  0.17831974]
 [ 0.         -2.86099568 -0.78429855]
 [ 0.          0.          0.        ]]
